In [ ]:
"""danielsinkin97@gmail.com"""

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, Button, VBox, HBox, Layout, interactive_output
import datetime
from pathlib import Path

from src.constants import FolderPath

# Global to hold current fig
current_fig = None


def plot_fourier(n_pos=3, n_neg=3):
    global current_fig

    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    N = 512
    x = np.linspace(0, 2 * np.pi, N, endpoint=False)
    f = np.full_like(x, 2, dtype=complex)

    for j in range(n_pos):
        freq = 1 + 2 * j
        coeff = 3 / (2**j)
        f += coeff * np.exp(1j * freq * x)

    for j in range(n_neg):
        freq = 1 + 2 * j
        coeff = 3 / (2**j)
        f += coeff * np.exp(-1j * freq * x)

    F = np.fft.fft(f)
    F_normalized = F / N
    F_shifted = np.fft.fftshift(F_normalized)

    dx = x[1] - x[0]
    freq_vals = np.fft.fftfreq(N, d=dx)
    omega = np.fft.fftshift(freq_vals * 2 * np.pi)

    axs[0].plot(x, f.real, label="Real part")
    axs[0].plot(x, f.imag, label="Imag part", linestyle="--")
    axs[0].set_title("Function f(x)")
    axs[0].set_xlabel("x")
    axs[0].set_ylabel("f(x)")
    axs[0].legend()
    axs[0].grid(True)

    axs[1].stem(omega, np.abs(F_shifted), basefmt=" ")
    axs[1].set_xlim(-10, 10)
    axs[1].set_title("Fourier Transform |F(ω)|")
    axs[1].set_xlabel("Angular frequency ω")
    axs[1].set_ylabel("Magnitude")
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

    current_fig = fig


def take_screenshot(b):
    global current_fig
    if current_fig is not None:
        Path("images").mkdir(exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = f"example_fourier_interactive_{timestamp}.png"
        current_fig.savefig(FolderPath.Images.joinpath(filepath), dpi=300)
        print(f"Screenshot saved as {filepath}")
    else:
        print("No figure to save.")


# Widgets
n_pos_slider = IntSlider(min=0, max=10, value=3, description="Positive Terms")
n_neg_slider = IntSlider(min=0, max=10, value=3, description="Negative Terms")
screenshot_button = Button(description="Take Screenshot", layout=Layout(width="150px"))
screenshot_button.on_click(take_screenshot)

# Hook up interactive output
output = interactive_output(
    plot_fourier, {"n_pos": n_pos_slider, "n_neg": n_neg_slider}
)

# Combine layout
ui = VBox([screenshot_button, HBox([n_pos_slider, n_neg_slider])])
display(VBox([ui, output]))